In [1]:
import threading, time

In [2]:
print('Start of program.')

Start of program.


In [3]:
def takeANap():
    time.sleep(5)
    print('Wake Up!')

In [4]:
threadObj = threading.Thread(target=takeANap)
threadObj.start()

In [5]:
print('End of program')

End of program
Wake Up!


### Passing Arguments to the Thread's Target Function

In [6]:
threadObj = threading.Thread(target=print, args=['Cats', 'Dogs', 'Frogs'], kwargs={'sep': ' & '})
threadObj.start()

Cats & Dogs & Frogs


# Project: Multithreading XKCD Downloader

### Step 1: Modify the Program to Use a Function

In [7]:
import requests, os, bs4

In [8]:
os.makedirs('xkcd', exist_ok=True)    # store comics in ./xkcd folder

In [14]:
def downloadXkcd(startComic, endComic):
    for urlNumber in range(startComic, endComic):
        #Download the page
        print('Downloading page https://xkcd/%s...' % (urlNumber))
        try:
            res = requests.get('https://xkcd.com/%s' % (urlNumber))
            res.raise_for_status()

            soup = bs4.BeautifulSoup(res.text, "html5lib")

            #Find the URL of the comic image.
            comicElem = soup.select('#comic img')
            if comicElem == []:
                print('Could not find comic image')
            else:
                comicUrl = 'http:' + comicElem[0].get('src')
                #Download the image
                print('Downloading image %s...' % (comicUrl))
                res = requests.get(comicUrl)
                res.raise_for_status()

                #Save the image to ./xkcd.
                imageFile = open(os.path.join('xkcd', os.path.basename(comicUrl)), 'wb')
                for chunk in res.iter_content(100000):
                    imageFile.write(chunk)
                imageFile.close()
        except requests.exceptions.HTTPError:
            pass

# Create and start Thread objects.
downloadThreads = []
for i in range(0, 200, 10):
    downloadThread = threading.Thread(target=downloadXkcd, args = (i, i + 9))
    downloadThreads.append(downloadThread)
    downloadThread.start()

# The main thread moves on as normal while the other threads we create download comics.
# Butr say there is some code you dont want to run in the main thread until all the threads have completed.
# Callin Thread object's join() method will block until that thread has finished.

for downloadThread in downloadThreads:
    downloadThread.join()

print('Done')








Done
